## Imports

In [4]:
# file:///C:/Users/Kream/Desktop/SmartImpedanceControl/ResearchPapers/3.pdf
import cv2
import numpy as np
from math import sqrt, floor, ceil
from IPython.display import clear_output
from threading import Thread

## Global Variables

In [18]:
# physics info
L1 = 70
L2 = 70

# window size 
width = 400
height = 400

center = (int(width/2), int(height/2))
theta1 = np.pi*1/4
theta2 = np.pi*1/4
motor1_angle = int(theta1*180/np.pi)
motor2_angle = int(theta2*180/np.pi)

# so motor angles are set automatically on launch since it'll be different than motor1_angle
current_motor1_angle = None
current_motor2_angle = None

## Kinematics

In [19]:
# inverse kin
# https://github.com/AymenHakim99/Forward-and-Inverse-Kinematics-for-2-DOF-Robotic-arm?fbclid=IwAR3Mu8nFWDik95ROO-O-ViZtPJ8EQK5ItO9Y9rz37tiFY2LDuernw1n67jM

import sympy as smp

# maybe we choose the signs base on which is closest? but also the one within possible range idk

x, y, a1, a2 = smp.symbols('x, y, a1, a2')

# defaults: sign1theta2 = 1  ;  sign2theta2 = 1
th2 = smp.acos( (x**2 + y**2 - a1**2 - a2**2) / (2*a1*a2) )
theta2_f = smp.lambdify( (x, y, a1, a2) , th2)


costh2, sinth2 = smp.symbols('costh2, sinth2')

th1 = smp.atan(y/(x+0.00001)) - smp.atan(a2*sinth2/(a1+a2*costh2))
theta1_f = smp.lambdify( (x, y, a1, a2, costh2, sinth2) , th1)


In [20]:
def inverseKin(px, py):
    
    # we gotta offset desired from the center
    px -= center[0]
    py -= center[1]
    
    # restricting arm to the maximum reach possible
    desired_circle = sqrt(px**2 + py**2)
    possible_circle = L1+L2 # maximum radius
    ratio = desired_circle / possible_circle
    
    if ratio > 1:
        if px < 0:
            px = ceil(px/ratio) # because floor for negative numbers is the opposite lmao
        else:
            px = floor(px/ratio)
            
        if py < 0:
            py = ceil(py/ratio) # because floor for negative numbers is the opposite lmao
        else:
            py = floor(py/ratio)
        
        desired_circle = sqrt(px**2 + py**2)
        possible_circle = sqrt((L1+L2)**2 + 0**2)
        ratio = desired_circle / possible_circle
        
    theta2 = theta2_f(py, px, L1, L2)
    theta1 = theta1_f(py, px, L1, L2, np.cos(theta2), np.sin(theta2))
    
    if py < 0:
        theta1 -= np.pi
    
    ogtheta1 = theta1
    ogtheta2 = theta2
        
    # trying to flip the approach to desired
    desired_point_tawila = sqrt(px**2+py**2)
    desired_point_angle = np.arccos(py/desired_point_tawila)
    if px < 0:
        desired_point_angle = - desired_point_angle
        
    theta1 += 2*(abs(theta1-desired_point_angle))
    theta2 = -theta2
    
    return theta1, theta2, ogtheta1, ogtheta2 # this function provides both approaches to the desired point so u can choose based on convenience

def forwardKin(theta1, theta2):
    x = L2*np.sin(theta1 + theta2) + L1*np.sin(theta1)
    y = L2*np.cos(theta1 + theta2) + L1*np.cos(theta1)
    return list((int(x) + center[0], int(y) + center[1]))

def forwardElbowKin(theta1):
    x = L1*np.sin(theta1)
    y = L1*np.cos(theta1)
    return list((int(x) + center[0], int(y) + center[1]))

def desiredIntroduced(event, x, y, flags, param):
    global desired, theta1, theta2, end_effector, elbow, ogend_effector, ogelbow, motor1_angle, motor2_angle
    if event==cv2.EVENT_LBUTTONDOWN: # or event==cv2.EVENT_MOUSEMOVE
        
        desired = [x, y]
        theta1, theta2, ogtheta1, ogtheta2 = inverseKin(x, y)  # this function provides both approaches to the desired point so u can choose based on convenience

        
        # BLUE ONE
        # limitation programming to be able to extend the arm past that 180 since the 2nd link can still turn more 
        theta1LimitReached = False
        if -np.pi < theta1 and theta1 < -np.pi/2:
            theta1 = -np.pi/2
            theta1LimitReached = True
        if theta1 > np.pi/2 or theta1 < -np.pi:
            theta1 = np.pi/2
            theta1LimitReached = True
            
        elbow = forwardElbowKin(theta1)
        
        # the case where theta1 reaches its limit
        if theta1LimitReached:
            centered_desired_x = x - center[0]
            centered_desired_y = y - center[1]
            
            centered_elbow_x = elbow[0] - center[0]
            centered_elbow_y = elbow[1] - center[1]
            
            tawila = sqrt((centered_desired_x-centered_elbow_x)**2 + (centered_desired_y-centered_elbow_y)**2)
            theta2 = np.pi - np.arccos((centered_desired_x-centered_elbow_x) / tawila)
            if y < center[1] and x < center[0]:
                theta2 = -theta2
            if y < center[1] and x > center[0]:
                theta2 = np.pi - theta2
            if y > center[1] and x > center[0]:
                theta2 = -np.pi +  theta2
        # TODO: find theta2 that lets the second link follow the desired point with the first link fixed at its limitation
        
        
        if theta2 < -np.pi/2:
            theta2 = -np.pi/2
        if theta2 > np.pi/2:
            theta2 = np.pi/2
        
        end_effector = forwardKin(theta1, theta2)

        # determine the angles of the motors based on angles of thetas
        deg_theta1 = theta1*180/np.pi
        temp_motor1_angle = int(deg_theta1+90)
        if -180 < temp_motor1_angle and temp_motor1_angle < 0:
            temp_motor1_angle = 0
        elif temp_motor1_angle < -180 or temp_motor1_angle > 180:
            temp_motor1_angle = 180
        
        deg_theta2 = theta2*180/np.pi
        motor1_angle, motor2_angle = [temp_motor1_angle, int(deg_theta2+90)] # i moved it to here just so they're both updated at once
        
        
        
        # GREEN ONE
        # limitation programming to be able to extend the arm past that 180 since the 2nd link can still turn more 
        ogtheta1LimitReached = False
        if -np.pi < ogtheta1 and ogtheta1 < -np.pi/2:
            ogtheta1 = -np.pi/2
            ogtheta1LimitReached = True
        if ogtheta1 > np.pi/2 or ogtheta1 < -np.pi:
            ogtheta1 = np.pi/2
            ogtheta1LimitReached = True
        
        ogelbow = forwardElbowKin(ogtheta1)
        
        # the case where theta1 reaches its limit
        if ogtheta1LimitReached:
            centered_desired_x = x - center[0]
            centered_desired_y = y - center[1]
            
            centered_ogelbow_x = ogelbow[0] - center[0]
            centered_ogelbow_y = ogelbow[1] - center[1]
            
            tawila = sqrt((centered_desired_x-centered_ogelbow_x)**2 + (centered_desired_y-centered_ogelbow_y)**2)
            ogtheta2 = np.pi - np.arccos((centered_desired_x-centered_ogelbow_x) / tawila)
            if y < center[1] and x < center[0]:
                ogtheta2 = -ogtheta2
            if y < center[1] and x > center[0]:
                ogtheta2 = np.pi - ogtheta2
        # TODO: find ogtheta2 that lets the second link follow the desired point with the first link fixed at its limitation
        
        ogend_effector = forwardKin(ogtheta1, ogtheta2)
        
end_effector = forwardKin(theta1, theta2)
elbow = forwardElbowKin(theta1)
ogend_effector = forwardKin(theta1, theta2)
ogelbow = forwardElbowKin(theta1)
desired = end_effector.copy()

## Communication

In [35]:
running = True
def communicationThread():
    global current_motor1_angle, current_motor2_angle, running
    while running:
        if motor1_angle != current_motor1_angle or motor2_angle != current_motor2_angle:
            # save the data
            current_motor1_angle = motor1_angle
            current_motor2_angle = motor2_angle
            
            clear_output(wait=True)
            print("Applying angles:", current_motor1_angle, "°", current_motor2_angle, "°")

        # whether we should stop the program or not
        cv2.waitKey(1)

## Gui

In [36]:
# allow the user to edit the desired point real time
cv2.destroyAllWindows()
cv2.namedWindow('output', cv2.WINDOW_AUTOSIZE)
cv2.setMouseCallback('output', desiredIntroduced)

def draw():
    window = np.ones((height, width, 3), dtype=np.uint8) #*255

    window = cv2.line(window, tuple(center), tuple(elbow), (255, 0, 0), 6)
    window = cv2.line(window, tuple(elbow), tuple(end_effector), (255, 0, 0), 6)
    #window = cv2.line(window, tuple(center), tuple(desired), (255, 255, 255), 6)
    
    #window = cv2.line(window, tuple(center), tuple(ogelbow), (0, 255, 0), 6)
    #window = cv2.line(window, tuple(ogelbow), tuple(ogend_effector), (0, 255, 0), 6)
    
    window = cv2.circle(window, tuple(desired), 5, (255, 255, 255), 2)   # we have casual coordinates but we want zero to be the middle of screen with a range of -4 to 4 just bcz it works with our other values
    return window

## Main

In [37]:
# start communication thread
thread = Thread(target=communicationThread)
thread.start()

# start main Gui thread
while True:
    window = draw()
    cv2.imshow('output', window)

    # whether we should stop the program or not
    if cv2.waitKey(1) & 0xFF == ord('q'):
        running = False
        thread.join()
        break

# clean up
cv2.destroyAllWindows()

Applying angles: 141 ° 6 °
